# Train and test set

### Feature groups
- raw Sentinel-2 bands
- indices based on raw Sentinel-2 bands (e.g. NDVI, NDWI, SAVI, EVI)

### More advanced ideas
- seasonal, quarterly and monthly time series
- min, max, median indices

## Setup

In [1]:
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/land_cover_classification_kaza

/content/drive/MyDrive/land_cover_classification_kaza


## Load and prepare raw data

We're using the labels provided by Nuno and the according Sentinel-2 bands.

In [4]:
raw_data = pd.read_csv('data/raw_data.csv')
raw_data.shape

(267442, 25)

In [5]:
raw_data

,system:index,B11_dry,B11_rainy,B12_dry,B12_rainy,B2_dry,B2_rainy,B3_dry,B3_rainy,B4_dry,...,B7_dry,B7_rainy,B8A_dry,B8A_rainy,B8_dry,B8_rainy,LC_Nr,LC_Out,Landcover,.geo
0,00000000000000000010_0,3025.0,2485.0,2112.0,1654.0,768.0,612.0,1012.0,880.0,1458.0,...,2192.0,2605.0,2540.0,2887.0,2513.0,2770.0,4,Active cro,Cropland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,00000000000000000010_1,3096.0,2522.0,2183.0,1632.0,776.0,595.0,1032.0,893.0,1472.0,...,2257.0,2596.0,2564.0,2872.0,2548.0,2684.0,4,Active cro,Cropland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,00000000000000000010_2,3096.0,2522.0,2183.0,1632.0,755.0,562.0,982.0,850.0,1416.0,...,2257.0,2596.0,2564.0,2872.0,2513.0,2744.0,4,Active cro,Cropland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,00000000000000000010_3,3047.0,2444.0,2084.0,1514.0,737.0,518.0,947.0,769.0,1392.0,...,2197.0,2555.0,2498.0,2837.0,2433.0,2680.0,4,Active cro,Cropland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,00000000000000000010_4,3047.0,2444.0,2084.0,1514.0,735.0,467.0,930.0,738.0,1368.0,...,2197.0,2555.0,2498.0,2837.0,2386.0,2624.0,4,Active cro,Cropland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267437,00000000000000000561_158,2436.0,1417.0,1898.0,793.0,505.0,378.0,608.0,471.0,742.0,...,1679.0,1657.0,2014.0,1885.0,1787.0,1560.0,8,Wetland,Wetland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
267438,00000000000000000561_159,2308.0,1442.0,1736.0,825.0,514.0,363.0,624.0,476.0,759.0,...,1627.0,1589.0,1977.0,1817.0,1786.0,1542.0,8,Wetland,Wetland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
267439,00000000000000000561_160,2308.0,1442.0,1736.0,825.0,532.0,348.0,625.0,493.0,776.0,...,1627.0,1589.0,1977.0,1817.0,1779.0,1560.0,8,Wetland,Wetland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
267440,00000000000000000561_161,2436.0,1417.0,1898.0,793.0,510.0,368.0,586.0,506.0,708.0,...,1679.0,1657.0,2014.0,1885.0,1814.0,1652.0,8,Wetland,Wetland,"{""geodesic"":false,""type"":""Point"",""coordinates""..."


In [6]:
raw_data = raw_data.drop(['system:index', '.geo'], axis=1)

In [7]:
# remove land cover class deforestation as it is not needed in this use case
raw_data = raw_data[raw_data['Landcover'] != 'Deforestation']
raw_data.shape

(261066, 23)

In [8]:
raw_data['Landcover'] = raw_data['Landcover'].str.capitalize()

## Downsample and balance raw data

In [9]:
raw_data['Landcover'].value_counts()

Forest      117277
Cropland     58759
Wetland      45501
Shrub        27552
Grass        10826
Bare           475
Water          376
Built up       300
Name: Landcover, dtype: int64

In [10]:
raw_data['LC_Nr'].value_counts()

7    117277
4     58759
8     45501
6     27552
5     10826
2       475
1       376
3       300
Name: LC_Nr, dtype: int64

In [11]:
def sample_train_and_test_data(df, land_cover_class, train_fraction, desired_train_samples=None):

  random_state = 42

  # filter and shuffle df
  df = df[df['Landcover'] == land_cover_class]
  df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

  # calculate n train samples
  n_train_samples = int(round(df.shape[0] * train_fraction, 0))

  # sample train and test samples
  train_samples = df.iloc[:n_train_samples]
  test_samples = df.iloc[n_train_samples:]

  # downsample train samples if specified
  if desired_train_samples is not None:
    train_samples = train_samples.sample(n=desired_train_samples, random_state=random_state).reset_index(drop=True)

  print(f'land cover class: {land_cover_class}; train samples: {train_samples.shape[0]}; test samples: {test_samples.shape[0]}')

  return train_samples, test_samples

In [12]:
train_fraction = 0.7
train_forest, test_forest = sample_train_and_test_data(raw_data, 'Forest', train_fraction, desired_train_samples=500)
train_cropland, test_cropland = sample_train_and_test_data(raw_data, 'Cropland', train_fraction, desired_train_samples=500)
train_wetland, test_wetland = sample_train_and_test_data(raw_data, 'Wetland', train_fraction, desired_train_samples=500)
train_shrub, test_shrub = sample_train_and_test_data(raw_data, 'Shrub', train_fraction, desired_train_samples=500)
train_grass, test_grass = sample_train_and_test_data(raw_data, 'Grass', train_fraction, desired_train_samples=500)
train_bare, test_bare = sample_train_and_test_data(raw_data, 'Bare', train_fraction)
train_water, test_water = sample_train_and_test_data(raw_data, 'Water', train_fraction)
train_built_up, test_built_up = sample_train_and_test_data(raw_data, 'Built up', train_fraction)

land cover class: Forest; train samples: 500; test samples: 35183
land cover class: Cropland; train samples: 500; test samples: 17628
land cover class: Wetland; train samples: 500; test samples: 13650
land cover class: Shrub; train samples: 500; test samples: 8266
land cover class: Grass; train samples: 500; test samples: 3248
land cover class: Bare; train samples: 332; test samples: 143
land cover class: Water; train samples: 263; test samples: 113
land cover class: Built up; train samples: 210; test samples: 90


In [13]:
train = pd.concat([train_forest, train_cropland, train_wetland, train_shrub, train_grass, train_bare, train_water, train_built_up], ignore_index=True)
train

,B11_dry,B11_rainy,B12_dry,B12_rainy,B2_dry,B2_rainy,B3_dry,B3_rainy,B4_dry,B4_rainy,...,B6_rainy,B7_dry,B7_rainy,B8A_dry,B8A_rainy,B8_dry,B8_rainy,LC_Nr,LC_Out,Landcover
0,1993.000000,2181.0,1200.0,1565.0,518.375000,612.0,692.500000,792.0,804.0,824.0,...,2164.0,1860.500000,2544.0,2196.0,2767.0,2250.666667,2670.0,7,Forest,Forest
1,1877.000000,1936.0,1243.0,947.0,405.000000,335.0,529.000000,556.0,555.0,336.0,...,2057.0,1768.000000,2450.0,2013.0,2755.0,1770.000000,2412.0,7,Forest,Forest
2,2244.500000,2038.0,1338.8,1057.0,478.000000,343.0,696.000000,612.0,739.5,460.0,...,2267.0,2123.333333,2638.0,2498.0,3081.0,2577.000000,2916.0,7,Forest,Forest
3,1872.500000,1748.0,1225.5,928.0,386.500000,294.0,522.000000,407.0,560.5,309.0,...,1709.0,1691.000000,1993.0,1934.5,2243.0,1749.000000,1964.0,7,Forest,Forest
4,2081.000000,1990.0,1229.0,996.0,433.000000,343.0,608.666667,602.0,628.0,398.0,...,2312.0,1978.000000,2623.0,2299.0,2914.0,2122.000000,2692.0,7,Forest,Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3300,3092.666667,3232.0,2524.5,2754.0,866.500000,1162.0,1096.000000,1492.0,1484.0,1278.0,...,2584.0,2031.000000,2969.0,2339.5,3306.0,2266.666667,3212.0,3,Built-up,Built up
3301,3586.000000,3501.0,3187.5,2601.0,1168.000000,1090.0,1326.000000,1250.0,2012.0,2154.0,...,2727.0,2563.500000,2979.0,2780.0,3186.0,2618.000000,2994.0,3,Built-up,Built up
3302,3474.400000,2904.0,2970.5,2308.0,1122.000000,1102.0,1458.000000,1466.0,1848.0,1642.0,...,2658.0,2521.000000,2922.0,2757.6,3097.0,2689.000000,2786.0,3,Built-up,Built up
3303,3447.000000,2056.0,3303.0,1414.0,1203.333333,576.0,1410.000000,784.0,1806.0,699.5,...,1892.5,2068.000000,2108.5,2315.0,2342.5,2502.000000,2186.0,3,Built-up,Built up


In [14]:
test = pd.concat([test_forest, test_cropland, test_wetland, test_shrub, test_grass, test_bare, test_water, test_built_up], ignore_index=True)
test

,B11_dry,B11_rainy,B12_dry,B12_rainy,B2_dry,B2_rainy,B3_dry,B3_rainy,B4_dry,B4_rainy,...,B6_rainy,B7_dry,B7_rainy,B8A_dry,B8A_rainy,B8_dry,B8_rainy,LC_Nr,LC_Out,Landcover
0,1466.250000,1610.0,747.800000,792.0,308.909091,325.0,455.000000,497.5,372.400000,310.0,...,2042.5,1723.5,2396.0,1914.333333,2568.5,1834.750000,2335.0,7,Forest,Forest
1,1355.000000,1421.0,657.000000,600.0,316.500000,228.0,447.000000,362.0,367.000000,224.0,...,1788.0,1798.5,2165.0,1992.500000,2436.0,1967.500000,2193.0,7,Forest,Forest
2,1879.500000,1911.0,1113.500000,935.0,390.000000,311.0,536.000000,539.0,563.666667,395.0,...,2221.0,1733.0,2521.0,2017.500000,2839.0,1758.000000,2472.0,7,Forest,Forest
3,1833.333333,1832.0,1062.000000,869.0,382.000000,304.0,531.333333,589.0,546.000000,309.0,...,2233.0,1788.4,2544.0,2055.250000,2756.0,1955.333333,2710.0,7,Forest,Forest
4,1906.142857,1685.0,1134.166667,934.0,399.400000,327.0,560.666667,532.0,654.142857,402.0,...,1958.0,1812.8,2284.0,2094.857143,2467.0,1778.000000,2142.0,7,Forest,Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78316,2897.000000,3235.0,2883.000000,3290.0,961.333333,1346.0,1237.000000,1580.0,1584.000000,1932.0,...,2261.0,2066.0,2315.0,2116.000000,2417.0,1868.000000,2188.0,3,Built-up,Built up
78317,3748.500000,2491.0,3817.500000,1922.5,1204.000000,830.0,1412.000000,1061.5,1762.000000,1065.0,...,2101.5,2339.5,2340.0,2625.000000,2498.0,2432.000000,2291.0,3,Built-up,Built up
78318,3840.000000,2655.5,3858.500000,2058.5,1327.000000,807.0,1559.000000,1053.0,1938.000000,1028.0,...,2190.5,2377.5,2410.5,2632.500000,2638.0,2644.000000,2422.0,3,Built-up,Built up
78319,3790.000000,3933.0,3551.000000,3485.0,1240.000000,1660.0,1376.000000,1972.0,1724.000000,2398.0,...,2708.0,2341.0,2977.0,2725.000000,3270.0,2559.000000,3160.0,3,Built-up,Built up


## Compute indices

In [15]:
def ndvi(df, nir_band, red_band):
  ndvi = (df[nir_band] - df[red_band]) / (df[nir_band] + df[red_band])
  return ndvi

In [16]:
def ndwi_mcf(df, green_band, nir_band):
  ndwi_mcf = (df[green_band] - df[nir_band]) / (df[green_band] + df[nir_band])
  return ndwi_mcf

In [17]:
def ndwi_gao(df, nir_band, swir_band):
  ndwi_gao = (df[nir_band] - df[swir_band]) / (df[nir_band] + df[swir_band])
  return ndwi_gao

In [18]:
def savi(df, nir_band, red_band):
  savi = (df[nir_band] - df[red_band]) / (df[nir_band] + df[red_band] + 0.5) * (1.0 + 0.5)
  return savi

In [19]:
def evi(df, nir_band, red_band, blue_band):
  evi = 2.5 * ((df[nir_band] - df[red_band]) / (df[nir_band] + 6 * df[red_band] - 7.5 * df[blue_band] + 1))
  return evi

In [20]:
train['NDVI_rainy'] = ndvi(train, 'B8_rainy', 'B4_rainy')
train['NDWI_MCF_rainy'] = ndwi_mcf(train, 'B3_rainy', 'B8_rainy')
train['NDWI_GAO_rainy'] = ndwi_gao(train, 'B8_rainy', 'B12_rainy')
train['SAVI_rainy'] = savi(train, 'B8_rainy', 'B4_rainy')
train['EVI_rainy'] = evi(train, 'B8_rainy', 'B4_rainy', 'B2_rainy')
train['NDVI_dry'] = ndvi(train, 'B8_dry', 'B4_dry')
train['NDWI_MCF_dry'] = ndwi_mcf(train, 'B3_dry', 'B8_dry')
train['NDWI_GAO_dry'] = ndwi_gao(train, 'B8_dry', 'B12_dry')
train['SAVI_dry'] = savi(train, 'B8_dry', 'B4_dry')
train['EVI_dry'] = evi(train, 'B8_dry', 'B4_dry', 'B2_dry')
train

,B11_dry,B11_rainy,B12_dry,B12_rainy,B2_dry,B2_rainy,B3_dry,B3_rainy,B4_dry,B4_rainy,...,NDVI_rainy,NDWI_MCF_rainy,NDWI_GAO_rainy,SAVI_rainy,EVI_rainy,NDVI_dry,NDWI_MCF_dry,NDWI_GAO_dry,SAVI_dry,EVI_dry
0,1993.000000,2181.0,1200.0,1565.0,518.375000,612.0,692.500000,792.0,804.0,824.0,...,0.528334,-0.542461,0.260921,0.792388,1.525620,0.473592,-0.529418,0.304482,0.710272,1.134514
1,1877.000000,1936.0,1243.0,947.0,405.000000,335.0,529.000000,556.0,555.0,336.0,...,0.755459,-0.625337,0.436142,1.132982,2.708062,0.522581,-0.539800,0.174909,0.783702,1.472014
2,2244.500000,2038.0,1338.8,1057.0,478.000000,343.0,696.000000,612.0,739.5,460.0,...,0.727488,-0.653061,0.467908,1.091071,1.977774,0.554048,-0.574702,0.316206,0.830947,1.339286
3,1872.500000,1748.0,1225.5,928.0,386.500000,294.0,522.000000,407.0,560.5,309.0,...,0.728113,-0.656685,0.358230,1.091929,2.563507,0.514614,-0.540291,0.175996,0.771753,1.341876
4,2081.000000,1990.0,1229.0,996.0,433.000000,343.0,608.666667,602.0,628.0,398.0,...,0.742395,-0.634487,0.459870,1.113412,2.286227,0.543273,-0.554199,0.266488,0.814761,1.412900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3300,3092.666667,3232.0,2524.5,2754.0,866.500000,1162.0,1096.000000,1492.0,1484.0,1278.0,...,0.430735,-0.365646,0.076768,0.646031,2.232225,0.208674,-0.348136,-0.053814,0.312969,0.418725
3301,3586.000000,3501.0,3187.5,2601.0,1168.000000,1090.0,1326.000000,1250.0,2012.0,2154.0,...,0.163170,-0.410933,0.070241,0.244731,0.271178,0.130886,-0.327586,-0.098097,0.196307,0.255438
3302,3474.400000,2904.0,2970.5,2308.0,1122.000000,1102.0,1458.000000,1466.0,1848.0,1642.0,...,0.258356,-0.310442,0.093836,0.387490,0.653864,0.185365,-0.296841,-0.049739,0.278017,0.392038
3303,3447.000000,2056.0,3303.0,1414.0,1203.333333,576.0,1410.000000,784.0,1806.0,699.5,...,0.515162,-0.472054,0.214444,0.772609,1.800509,0.161560,-0.279141,-0.137984,0.242312,0.403338


In [21]:
test['NDVI_rainy'] = ndvi(test, 'B8_rainy', 'B4_rainy')
test['NDWI_MCF_rainy'] = ndwi_mcf(test, 'B3_rainy', 'B8_rainy')
test['NDWI_GAO_rainy'] = ndwi_gao(test, 'B8_rainy', 'B12_rainy')
test['SAVI_rainy'] = savi(test, 'B8_rainy', 'B4_rainy')
test['EVI_rainy'] = evi(test, 'B8_rainy', 'B4_rainy', 'B2_rainy')
test['NDVI_dry'] = ndvi(test, 'B8_dry', 'B4_dry')
test['NDWI_MCF_dry'] = ndwi_mcf(test, 'B3_dry', 'B8_dry')
test['NDWI_GAO_dry'] = ndwi_gao(test, 'B8_dry', 'B12_dry')
test['SAVI_dry'] = savi(test, 'B8_dry', 'B4_dry')
test['EVI_dry'] = evi(test, 'B8_dry', 'B4_dry', 'B2_dry')
test

,B11_dry,B11_rainy,B12_dry,B12_rainy,B2_dry,B2_rainy,B3_dry,B3_rainy,B4_dry,B4_rainy,...,NDVI_rainy,NDWI_MCF_rainy,NDWI_GAO_rainy,SAVI_rainy,EVI_rainy,NDVI_dry,NDWI_MCF_dry,NDWI_GAO_dry,SAVI_dry,EVI_dry
0,1466.250000,1610.0,747.800000,792.0,308.909091,325.0,455.000000,497.5,372.400000,310.0,...,0.765595,-0.648720,0.493444,1.148176,2.878874,0.662551,-0.602577,0.420882,0.993602,2.085102
1,1355.000000,1421.0,657.000000,600.0,316.500000,228.0,447.000000,362.0,367.000000,224.0,...,0.814646,-0.716634,0.570354,1.221717,2.692834,0.685586,-0.629737,0.499333,1.028158,2.226938
2,1879.500000,1911.0,1113.500000,935.0,390.000000,311.0,536.000000,539.0,563.666667,395.0,...,0.724451,-0.641979,0.451130,1.086486,2.068313,0.514429,-0.532694,0.224447,0.771478,1.347398
3,1833.333333,1832.0,1062.000000,869.0,382.000000,304.0,531.333333,589.0,546.000000,309.0,...,0.795296,-0.642922,0.514389,1.192747,2.626915,0.563433,-0.572654,0.296067,0.844980,1.488313
4,1906.142857,1685.0,1134.166667,934.0,399.400000,327.0,560.666667,532.0,654.142857,402.0,...,0.683962,-0.602094,0.392718,1.025742,2.068966,0.462085,-0.520525,0.221084,0.692985,1.037397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78316,2897.000000,3235.0,2883.000000,3290.0,961.333333,1346.0,1237.000000,1580.0,1584.000000,1932.0,...,0.062136,-0.161359,-0.201168,0.093193,0.173630,0.082271,-0.203221,-0.213639,0.123389,0.170550
78317,3748.500000,2491.0,3817.500000,1922.5,1204.000000,830.0,1412.000000,1061.5,1762.000000,1065.0,...,0.365316,-0.366741,0.087457,0.547892,1.247456,0.159752,-0.265349,-0.221698,0.239599,0.421384
78318,3840.000000,2655.5,3858.500000,2058.5,1327.000000,807.0,1559.000000,1053.0,1938.000000,1028.0,...,0.404058,-0.393957,0.081129,0.605999,1.372858,0.154081,-0.258149,-0.186774,0.231097,0.408518
78319,3790.000000,3933.0,3551.000000,3485.0,1240.000000,1660.0,1376.000000,1972.0,1724.000000,2398.0,...,0.137100,-0.231489,-0.048909,0.205631,0.373603,0.194957,-0.300635,-0.162357,0.292401,0.579218


## Save train and test set

In [22]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)